# **Importando Bibliotecas**

In [2]:
!pip install pdfplumber -q
!pip install xlsxwriter

In [3]:
import pdfplumber
import os
import pandas as pd
import xlsxwriter
from pprint import pprint

# **Funcoes**

In [4]:
def dadosPagina(pdf,pagina):
    page = pdf.pages[pagina]
    head = page.extract_text_simple()
    head = head.split('\n')
    cabecalho = []
    table = page.extract_tables()
    return head,table

def cabecalho(pdf,pagina):
    page = pdf.pages[pagina]
    head = page.extract_text_simple()
    head = head.split('\n')
    return head


# **Agendamento**

In [5]:
pdf = pdfplumber.open('/content/Relatório.pdf')
qtdPages = len(pdf.pages)

In [9]:
#Pegando o numero de pagina por profissional

profPaginas = {}

paginas = []

for pagina in range(0,qtdPages):
    head,table = dadosPagina(pdf,pagina)
    if "Profissional" in head[5]:
        paginas.append(pagina)
        profPaginas[head[5][14:]] = pagina

contador = 0
for prof, page in profPaginas.items():
    pages = []
    if contador <= len(paginas):
        try:
            for x in range(paginas[contador],paginas[contador+1]):
                pages.append(x)
            contador += 1
        except:
            for x in range(profPaginas[prof],qtdPages):
                pages.append(x)

    profPaginas[prof] = pages

profPaginas

{'Andre Yunes Perim': [0],
 'Carolina Barauna Assumpcao': [1, 2],
 'Cesar Piedade Damasio': [3],
 'Daniel Guidi Ferrari': [4],
 'Gabriel de Sousa Brum Marques Almeida': [5],
 'Jose Luis Teixeira Filho': [6],
 'Leila Passos Rossini': [7, 8],
 'Luiz Felipe Diniz Caruso': [9],
 'Marcia Maria Brunetta Dias': [10, 11],
 'Marcos Broglio': [12],
 'Nelson Dax da Silva Filho': [13],
 'Rodrigo Bitun': [14],
 'Sergio Alexandre da Silva Meleiro': [15],
 'Thiago Ricardo Roessle': [16]}

In [7]:
profPacientes = {}


for prof, paginas in profPaginas.items():
    linhas = []
    for pagina in paginas:
        head,table = dadosPagina(pdf,pagina)
        if len(table) > 1:
            for tabela in range(0,len(table)):
                if pagina == paginas[0]:
                    if tabela == 0:
                        for linha in table[tabela][1:]:
                            linhas.append(linha)
                    else:
                        for linha in table[tabela][0:]:
                            linhas.append(linha)
                elif pagina != paginas[0]:
                    for linha in table[tabela][0:]:
                        linhas.append(linha)
        elif len(table) == 1:
            if pagina == paginas[0]:
                for linha in table[0][1:]:
                    linhas.append(linha)
            elif pagina != paginas[0]:
                for linha in table[0][0:]:
                    linhas.append(linha)

    profPacientes[prof] = linhas

#Convertendo a coluna de prontuarios para o formato INT:
for prof, linha in profPacientes.items():
    for x in range(0,len(linha)):
        prontuario = profPacientes[prof][x][3]
        try:
            profPacientes[prof][x][3] = int(prontuario)
        except:
            pass

#Ordenando a coluna prontuarios em ordem crescente
for prof, linha in profPacientes.items():
    temp = {}
    sortedTemp = {}
    contador = 1
    for x in range(0,len(linha)):
        prontuario = profPacientes[prof][x][3]
        if type(prontuario) == type('teste'):
            prontuario = contador
            contador = contador + 1
        temp[prontuario] = linha[x]
    for x in sorted(temp):
        sortedTemp[x] = temp[x]
    profPacientes[prof] = sortedTemp.values()

In [10]:
workbook = xlsxwriter.Workbook("Agendamento.xlsx")
cell_format1 = workbook.add_format()
cell_format2 = workbook.add_format()
cell_format3 = workbook.add_format()
tamanho_col = {1:173.68,2:85.41,3:86.55,4:175,5:99.04,6:175,7:80.125,8:133}
pagina = 0
colunas = ["Horário", "Paciente", "Data Nasc.", "Prontuário", "Exames",
            "Telefone", "Observação", "CNS", "Seq. Atendimento",]

for prof, pacientes in profPacientes.items():
    contador = 10
    contador2 = 6
    contador3 = 0
    head = []


    if len(pacientes) <= 14:
        worksheet = workbook.add_worksheet(f"Pagina {pagina}")
        head = cabecalho(pdf,profPaginas[prof][0])

        #Inserindo o cabecalho* da pagina
        for linha in range(0,8):
            cell_format1.set_bold()
            cell_format1.set_font_size(12)
            worksheet.write(linha,0,head[linha],cell_format1)

        for linha in range(0,len(pacientes)):
            cell_format2.set_align('justify')
            cell_format2.set_align('vjustify')
            cell_format2.set_border(1)
            value = list(profPacientes[prof])
            for coluna in [0,1,2,3,4,5,6,7,8]:
                worksheet.write(contador,coluna,value[linha][coluna],cell_format2)

                if coluna == 5:
                    if (value[linha][1].count('\n') > value[linha][5].count('\n')
                        and value[linha][1].count('\n') > value[linha][7].count('\n')):
                        worksheet.set_row(contador,25)
                    elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                        if value[linha][5].count('\n') == 1:
                            worksheet.set_row(contador,25)
                        elif value[linha][5].count('\n') == 2:
                            worksheet.set_row(contador,35)
                        elif value[linha][5].count('\n') >= 3:
                            worksheet.set_row(contador,48)
                    elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                        worksheet.set_row(contador,26)
                    elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                        worksheet.set_row(contador,26)

            contador += 1
        worksheet.set_landscape()
        worksheet.print_area('A1:I28')
        worksheet.set_print_scale(90)
        worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)

        for coluna in [0,1,2,3,4,5,6,7,8]:
            cell_format3.set_bg_color('#B4B4B4')
            cell_format3.set_bold()
            cell_format3.set_font_size(12)
            worksheet.write(9,coluna,colunas[coluna],cell_format3)

        #Alterando o tamanho das colunas:
        for key, value in tamanho_col.items():
            worksheet.set_column_pixels(key,key,value)

        #Inserindo o final da planilha.
        worksheet.write(26,0,head[-2],cell_format1)
        worksheet.write(27,0,head[-1],cell_format1)

        pagina += 1

    elif len(profPacientes[prof]) > 14:

        head = cabecalho(pdf,profPaginas[prof][0])
        tabelas = []

        lista = list(profPacientes[prof])
        bool = True

        while bool:
            temp = []
            if len(lista) >= 14:
                for num in range(0,14):
                    temp.append(lista[num])
                tabelas.append(temp)
                del lista[0:14]
            elif len(lista) < 14:
                for num in range(0,len(lista)):
                    temp.append(lista[num])
                if len(temp) == 0:
                    pass
                else:
                    tabelas.append(temp)
                    del lista[0:len(lista)]
                bool = False


        for tabela in range(0,len(tabelas)):

            if tabela == 0:
                worksheet = workbook.add_worksheet(f"Pagina {pagina}")
                head = cabecalho(pdf,profPaginas[prof][0])

                #Inserindo o cabecalho* da pagina
                for linha in range(0,8):
                    cell_format1.set_bold()
                    cell_format1.set_font_size(12)
                    worksheet.write(linha,0,head[linha],cell_format1)

                for linha in range(0,14):
                    cell_format2.set_align('justify')
                    cell_format2.set_align('vjustify')
                    cell_format2.set_border(1)
                    value = tabelas[tabela]

                    for coluna in [0,1,2,3,4,5,6,7,8]:
                        worksheet.write(contador,coluna,value[linha][coluna],cell_format2)

                        if coluna == 5:
                            if (value[linha][1].count('\n') > value[linha][5].count('\n')
                                and value[linha][1].count('\n') > value[linha][7].count('\n')):
                                worksheet.set_row(contador,25)
                            elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                                if value[linha][5].count('\n') == 1:
                                    worksheet.set_row(contador,25)
                                elif value[linha][5].count('\n') == 2:
                                    worksheet.set_row(contador,35)
                                elif value[linha][5].count('\n') >= 3:
                                    worksheet.set_row(contador,48)
                            elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                                worksheet.set_row(contador,26)
                            elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                                worksheet.set_row(contador,26)

                    contador += 1

                worksheet.set_landscape()
                worksheet.print_area('A1:I28')
                worksheet.set_print_scale(90)
                worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)

                for coluna in [0,1,2,3,4,5,6,7,8]:
                    cell_format3.set_bg_color('#B4B4B4')
                    cell_format3.set_bold()
                    cell_format3.set_font_size(12)
                    worksheet.write(9,coluna,colunas[coluna],cell_format3)

                #Alterando o tamanho das colunas:
                for key, value in tamanho_col.items():
                    worksheet.set_column_pixels(key,key,value)

                #Inserindo o final da planilha.
                worksheet.write(26,0,head[-2],cell_format1)
                worksheet.write(27,0,head[-1],cell_format1)

                pagina = pagina + 1

            elif tabela != 0:
                worksheet = workbook.add_worksheet(f"Pagina {pagina}")
                head = cabecalho(pdf,profPaginas[prof][0])
                contador2 = 6

                for linha in range(0,4):
                    worksheet.write(linha,0,head[linha],cell_format1)

                for linha in range(0,len(tabelas[tabela])):
                    cell_format2.set_align('justify')
                    cell_format2.set_align('vjustify')
                    cell_format2.set_border(1)
                    value = tabelas[tabela]

                    for coluna in [0,1,2,3,4,5,6,7,8]:
                        worksheet.write(contador2,coluna,value[linha][coluna],cell_format2)

                        if coluna == 5:
                            if (value[linha][1].count('\n') > value[linha][5].count('\n')
                                and value[linha][1].count('\n') > value[linha][7].count('\n')):
                                worksheet.set_row(contador2,25)
                            elif value[linha][5].count('\n') > value[linha][7].count('\n'):
                                if value[linha][5].count('\n') == 1:
                                    worksheet.set_row(contador2,25)
                                elif value[linha][5].count('\n') == 2:
                                    worksheet.set_row(contador2,35)
                                elif value[linha][5].count('\n') >= 3:
                                    worksheet.set_row(contador2,48)
                            elif value[linha][7].count('\n') > value[linha][5].count('\n'):
                                worksheet.set_row(contador2,26)
                            elif value[linha][7].count('\n') == 1 and value[linha][5].count('\n') == 1:
                                worksheet.set_row(contador2,26)

                    contador2 += 1

                worksheet.set_landscape()
                worksheet.print_area('A1:I28')
                worksheet.set_print_scale(90)
                worksheet.set_margins(left=0.20, right=0.10, top=0.15, bottom=0.15)

                for coluna in [0,1,2,3,4,5,6,7,8]:
                    cell_format3.set_bg_color('#B4B4B4')
                    cell_format3.set_bold()
                    cell_format3.set_font_size(12)
                    worksheet.write(5,coluna,colunas[coluna],cell_format3)

                #Alterando o tamanho das colunas:
                for key, value in tamanho_col.items():
                    worksheet.set_column_pixels(key,key,value)

                #Inserindo o final da planilha.
                worksheet.write(26,0,head[-2],cell_format1)
                worksheet.write(27,0,head[-1],cell_format1)


                pagina = pagina + 1

workbook.close()